# **Actividad 03**
----
> **Alumna :** Claudia Luz Rojas Soto 

> **Asignatura:** Minería de Datos         
                               
> **Docente:** Carlos Fernando Montoya Cubas

## **Instalación de Librerías**
---

In [1]:
import numpy as np  
import pandas as pd  
import itertools

## **Módulos a necesitar**
---

***findsubsets(C,n):*** Recibe como parámetros C que es conjunto de datos de los cuales queremos generar combinaciones de sus elementos y n que es la cantidad de elementos de cada combinación.

In [2]:
# Importamos funciones de la libreria itertools que permiten realizar las combinaciones
from itertools import chain, combinations
def findsubsets(C,n): 
    # Devolvemos un conjunto de combinaciones de n elementos
    return set(itertools.combinations(C,n))

***print_rules(rules):*** Recibe como parámetro un arreglo de reglas y las imprime con un formato preestablecido mostrando el soporte, la confianza y el sustento(lift).

In [3]:
def print_rules(rules):
    # Dentro de cada arreglo de reglas encontraremos una regla con la estructura:
    # [ index , antecedente, consecuente, support, confidence, lift]
    for rule in rules:
        # Mostrando estos valores con la siguiente estructura
        print(" ======= REGLA DE ASOCIACIÓN =======")
        if(str(rule[0])==1):
            print(str(rule[0]),'--------',rule[1])
        else:
            print(rule[0],'--------',rule[1])
        print('SUPPORT    = ',rule[2])
        print('CONFIDENCE = ',rule[3])
        print('LIFT       = ',rule[4])

## **Datos**
---

### **Lectura de Datos**
---
***spotify.npy:*** Este conjunto de datos contiene distintas playlists compuestas por distintas canciones. Leemos este conjunto de datos volviendolo una lista de lista.

In [4]:
data = np.load('spotify.npy', allow_pickle=True)
data = data.tolist()
# Convertimos la data a una lista de listas
playlists = [data[i] for i in data ]

***Tamaño del dataset:***

In [5]:
len(playlists)

10000

***Primer item:***


In [6]:
# Mostramos un playlist del conjunto de datos
print(playlists[0:1])
# Mostramos el tamaño de este playlist
print('\n El tamaño de la lista es:',len(playlists[0:1][0]))

[['Lose Control (feat. Ciara & Fat Man Scoop)', 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix', 'Promiscuous', 'Right Where You Want Me - Radio Edit Version', 'Beautiful Soul', "Leavin'", 'Me & U', 'Ice Box', 'Sk8er Boi', 'Run It!', 'Check On It - feat. Bun B and Slim Thug', "Jumpin', Jumpin'", 'Soak Up The Sun', 'Where Is The Love?', "Stacy's Mom", 'Just The Girl', 'Yo (Excuse Me Miss)', 'Year 3000', 'Lip Gloss', 'Everytime We Touch - Radio Edit', 'Whatcha Say', 'Miss Independent', 'Party In The U.S.A.', 'The Great Escape', 'Replay', 'Forever', 'Your Love Is My Drug', 'Closer', 'One Less Lonely Girl', 'Paper Planes', 'Mr. Brightside', 'All The Small Things', 'Beep', 'Somebody To Love', 'Dirty Little Secret', 'Baby', 'A Thousand Miles', 'Livin on Sunday', 'See You Again', 'How Do You Sleep? - Featuring Ludacris', 'This Is Me', 'My Happy Ending', 'Check Yes Juliet', 'The Great Escape']]

 El tamañ

### **Declaración de parametros**
---
- ***veces_minimas_aparece_en_lista:***  100
- ***soporte_min:*** veces_minimas_aparece_en_lista / cantidad de playlists
- ***confianza_min:*** 0.5
- ***sustento_min:*** 1.5

In [7]:
veces_minimas_aparece_en_lista=100
soporte_min = veces_minimas_aparece_en_lista / len(playlists)
confianza_min =0.5
sustento_min =1.5

print("Soporte:",soporte_min)

Soporte: 0.01


## **Implementar el algoritmo Apriori**
---

### **Módulo get_frequent_itemsets**
---
#### Input: 
- *playlists:* Arreglo de arreglos, donde cada uno de estos arreglos representa un conjunto de canciones.
- *min_support:* Umbral de soporte.

#### Output:
- *count_item:* Diccionario con itemsets frecuentes, y sus valores support, para cada uno de estos.


In [8]:
def get_frequent_itemsets(playlists, min_support):
    # Diccionario que guardara nuestros itemsets frecuentes y su soporte
    count_item= dict()
    # Base: 1-itemset
    # Recorremos cada playlist en el conjunto de playlist
    for playlist in playlists:
        # Recorremos cada canción del playlist
        for song in playlist:
            # Si la cancion no esta en nuestro diccionario, lo agregamos y empezamos a recuperar el numero de apariciones en 1
            if (not song in count_item):
                count_item[song] = 1
            # Si la canción ya está en nuestro diccionario, incrementamos el número de apariciones de este en uno
            else:
                count_item[song] = count_item[song] + 1
    # Luego de obtener el numero de apariciones de cada item de nuestro diccionario
    # Calculamos el soporte de estos, y validamos si pasa el min_support
    # En caso no pase el umbral, sacamos el item del diccionario
    for item in list(count_item): 
        count_item[item] = count_item[item]/len(playlists)
        if(count_item[item] < min_support):
            count_item.pop(item)
    
    # Obtención de k-itemsets
    k = 2 # Iniciamos en 2-itemsets
    pos = 0 
    bandera = True # Bandera que nos indicara si ya se obtuvieron todos los itemsets frecuentes
    # Si la bandera es false termina el bucle
    while(bandera):
        # Recuperamos los keys del diccionario en una lista
        lista = list(count_item)
        # Nueva lista : Lista auxiliar que guardará los posibles nuevos itemsets frecuentes
        nueva_lista = []
        # Recorremos los items que existen en la lista y los emparejamos con el resto de su forma (de manera similar a una matriz)
        # Para esto será necerario guardar la posición de la ultimo k itemset obtenido
        # Este pos nos permitira recorrer sectores de la lista
        # Por ejemplo, en el caso de la forma 2-itemset frec., el recorrido de la lista empezaria cuando acaben los 1-itemsets frec. 
        # Y así sucesivamente
        for i in range(pos,len(lista)):
            for j in range(i+1,len(lista)):
                # Para obtener posibles itemsets tenemos dos casos, que sea 2-itemsets o k-itemsets.
                if(k==2):
                    # En el caso de que estemos generando 2-itemsets, mezclamos todos los 1-itemsets 
                    nuevo_item = set()
                    nuevo_item.add(lista[i])
                    nuevo_item.add(lista[j]) 
                    nuevo_item = list(nuevo_item)
                    if(len(nuevo_item)==k):
                        # Agregamos el nuevo item ordenado en la lista auxiliar
                        nueva_lista.append(tuple(sorted(nuevo_item)))
                else:
                    # En el caso de los k-itemsets
                    # Un candidato es la mezcla de los k-1 itemsets que tengan los mismos k-2 primeros items
                    if(lista[i][:k-2]==lista[j][:k-2]):
                        # En caso se cumpla esto, realizamos las union de los k-1 itemsets, y se agrega el candidato (ordenado) a la lista auxiliar
                        nuevo=list(set(lista[i]).union(set(lista[j])))
                        if not nuevo in nueva_lista:
                            nueva_lista.append(tuple(sorted(nuevo)))
        
        # Recuperamos la posición donde se encuentra el primer k-1 itemset
        pos = len(lista)

        # Si la lista auxiliar tiene candidatos a itemsets frecuentes
        # Los añadimos al diccionario inicialmente creado
        if(len(nueva_lista)!=0):
            # Por cada item candidato en la lista auxiliar
            for item_nuevo in nueva_lista:
                # ----- FALTA VALIDACIÓN
                # Buscamos si es parte de algun playlist
                for playlist in playlists: 
                    # Realizamos esto con la intersección de conjuntos
                    if(set(item_nuevo).intersection(playlist) == set(item_nuevo)):
                        # Si el item set candidato se encuentra en algun playlist lo agregamos al diccionario con un value de 1.
                        if (not item_nuevo in count_item.keys()):
                            count_item[item_nuevo] = 1
                        # Caso contrario, incrementamos el value del itemset candidato
                        else:
                            count_item[item_nuevo] = count_item[item_nuevo] + 1 
                # Obtenemos el support del itemset agregado (si este ha sido agregado)
                if (item_nuevo in count_item.keys()):
                    count_item[item_nuevo] = count_item[item_nuevo]/len(playlists)
                    # En caso no pase el umbral de min_support lo eliminamos del diccionario 
                    if count_item[item_nuevo] < min_support:
                        count_item.pop(item_nuevo)
        #Caso contrario, volvemos la bandera a false, ya que no se podran obtener más itemsets frecuentes.
        else:
            bandera=False
        # Agregamos 1 al tamaño del itemset que se quiere generar
        k = k+1
        # Generamos...
    # Guardamos los valores del diccionario de la forma [itemset_frecuente, support] en un archivo .csv
    df=pd.DataFrame.from_dict(count_item,orient="index")
    df.to_csv('./playlists-'+str(len(playlists))+'-'+str(min_support)+'.csv')
    return count_item

### **Módulo generate_association_rules**
---
Input: 
- *frequent_itemsets:* Diccionario de la forma donde cada elemento es de la forma: **(itemset frecuente: support)**.
- *confidence:* Umbral de confianza, default = 0.
- *lift:* Umbral de lift, default = 0.

Output:
- *rules:* Arreglo de reglas que guardan el antecedente, consecuente, support, confidence y lift de cada una de estas.

In [9]:
from ast import literal_eval

def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):   
    # Single_tam es la posicion donde esta el ultimo 1-itemset
    single_tam = 0
    # Para obtener este valor recorremos las keys del diccionario
    items = list(frequent_itemsets)
    values = list(frequent_itemsets.values())
    # Existe un problema al leer el diccionario desde un .csv ya que no lee tuplas si no strings
    # La siguiente parte de codigo crea un diccionario auxiliar que nos permitira eliminar este error
    aux_dict = dict()
    for i in range(len(items)):
        val = items[i]
        if val.startswith("("):
            val = literal_eval(val)
            items[i] = val
            aux_dict[val] = values[i]
        else:
            single_tam = single_tam + 1
            aux_dict[items[i]] = values[i]
    # Recuperamos las keys del diccionario de itemsets frecuentes
    rules = []
    # Recorremos este arreglo de atras para adelante, es decir, desde los itemsets con k elementos hasta los itemsets con 2 elementos, ayudandonos del single_tam anterior.
    for i in range(len(items)-1,single_tam,-1):
        # Recuperamos el tamaño del item
        tam_item = len(items[i])
        # Para generar las reglas empezamos a partir de un tamaño de tam_item - 1
        tam_antecedente = tam_item - 1
        # Mientras el tamaño del antecedente sea mayor que uno buscamos reglas
        while tam_antecedente>=1:
            # Generamos combinaciones a partir de los elementos de itemset frecuente con el tamaño del antecedente, predefinido anteriormente
            subconjuntos = list(findsubsets(items[i],tam_antecedente))
            # Recogemos el valor del support del item analizado
            x_u_y = aux_dict[items[i]]
            # Para cada combinación con k (tam_antecedente) items
            for elemento in subconjuntos:
                # -FALTA PODA CONFIDENCE 
                # Recogemos el valor del support del antecedente      
                if (len(elemento) == 1): 
                    x = aux_dict[str(elemento[0])] # Elemento[0] lo realizamos para recogerlo como string ya que solo es 1 elemento.
                else:
                    x = aux_dict[elemento]
                # Obtenemos el consecuente
                item_diff = tuple(sorted(list(set(items[i]).difference(elemento))))
                # Si es solo un valor, lo recogemos como string para buscarlos
                if(len(item_diff)==1):
                    item_diff = list(item_diff)[0]
                
                # Obtenemos el valor del confidence dado por:
                # c( x -> y ) = sup(x u y) / sup ( x )
                confidence_ob = x_u_y/x
                # l(x -> y) = confidence / sup (y)
                # sup (y) = frequent_itemsets[item_diff]
                lift_ob = confidence_ob/aux_dict[item_diff]
                # Si los valores obtenidos pasan el umbral, agregamos la regla a nuestro arreglo de reglas con un index.
                if (confidence_ob>= confidence and lift_ob>= lift):
                    # Si el elemento agregado es solo uno lo volvemos string
                    if(len(elemento)==1):
                        rules.append([elemento[0],item_diff,x_u_y,confidence_ob,lift_ob])
                    # Caso contrario continua como tupla
                    else:
                        rules.append([elemento,item_diff,x_u_y,confidence_ob,lift_ob])
            # Reducimos el tamaño del antecedente, para generar las combinaciones con este tamaño
            tam_antecedente = tam_antecedente - 1
    return rules



## **Aplicar el algoritmo y obtener reglas de asociación**
---

La generación de los datasets encontrados en la carpeta experiments fueron realizados en paralelo en pc's. Por lo que se optó por guardar los resultados en archivos .csv para su mejor lectura. Las funciones usadas fueron las siguientes:

In [ ]:
# 10000 playlists
get_frequent_itemsets(playlists,0.01) # 10000 playlists, support: 0.01
get_frequent_itemsets(playlists,0.015) # 10000 playlists, support: 0.015
get_frequent_itemsets(playlists,0.02) # 10000 playlists, support: 0.02
get_frequent_itemsets(playlists,0.025) # 10000 playlists, support: 0.025
# 5000 playlists
get_frequent_itemsets(playlists[:5000],0.01) # 5000 playlists, support: 0.01
get_frequent_itemsets(playlists[:5000],0.02) # 5000 playlists, support: 0.02
# 2500 playlists
get_frequent_itemsets(playlists[:2500],0.01) # 2500 playlists, support: 0.01
get_frequent_itemsets(playlists[:2500],0.02) # 2500 playlists, support: 0.02
# 1250 playlists
get_frequent_itemsets(playlists[:1250],0.01) # 1250 playlists, support: 0.01
get_frequent_itemsets(playlists[:1250],0.02) # 1250 playlists, support: 0.02

Siendo los archivos .csv , los siguientes:
 - playlists-10000-0.01.csv
 - playlists-10000-0.015.csv
 - playlists-10000-0.02.csv 
 - playlists-10000-0.025.csv
 - playlists-5000-0.01.csv
 - playlists-5000-0.02.csv
 - playlists-2500-0.01.csv
 - playlists-2500-0.02.csv
 - playlists-1250-0.01.csv
 - playlists-1250-0.02.csv

Identificados con la palabra 'playlists', el tamaño del dataset tomado, y el valor siguiente el min-support con el que fue evaluado este dataset.


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-10000-0.01.csv', los cuales fueron obtenidos de las 10000 primeras canciones con un support de 0.01. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [10]:
frequent_itemsets_10000_01 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-10000-0.01.csv', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_10000_01.pop(list(frequent_itemsets_10000_01.keys())[0])
rules_10000_01 = generate_association_rules(frequent_itemsets_10000_01,0.5,1.2)
rules_pd_10000_01 = pd.DataFrame(rules_10000_01,columns = ["Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_10000_01= rules_pd_10000_01.sort_values('Lift',ascending=False)
rules_pd_10000_01.to_csv("rules_10000_01.csv")
rules_pd_10000_01.head(10)

,Antecedente,Consecuente,Support,Confidence,Lift
92,No Heart,X (feat. Future),0.0103,0.691275,33.720740
91,X (feat. Future),No Heart,0.0103,0.502439,33.720740
105,Knee Deep (feat. Jimmy Buffett),Chicken Fried,0.0107,0.656442,30.112005
107,You Was Right,Money Longer,0.0117,0.587940,29.844655
106,Money Longer,You Was Right,0.0117,0.593909,29.844655
56,Butterfly Effect,Bank Account,0.0111,0.572165,24.769045
83,Magnolia,Bank Account,0.0113,0.565000,24.458874
8,"(HUMBLE., Tunnel Vision)",XO TOUR Llif3,0.0102,0.733813,22.236756
31,"(Bounce Back, Broccoli (feat. Lil Yachty))",Bad and Boujee (feat. Lil Uzi Vert),0.0100,0.775194,21.960164
46,"(HUMBLE., XO TOUR Llif3)",DNA.,0.0102,0.500000,21.551724


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-10000-0.015.csv', los cuales fueron obtenidos de las 10000 primeras canciones con un support de 0.015. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [11]:
frequent_itemsets_10000_015 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-10000-0.015.csv', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_10000_015.pop(list(frequent_itemsets_10000_015.keys())[0])
rules_10000_015 = generate_association_rules(frequent_itemsets_10000_015,0.5,1.2)
rules_pd_10000_015 = pd.DataFrame(rules_10000_015,columns = ["Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_10000_015= rules_pd_10000_015.sort_values('Lift',ascending=False)
rules_pd_10000_015.to_csv("rules_10000_01.csv")
rules_pd_10000_015.head(10)

,Antecedente,Consecuente,Support,Confidence,Lift
6,DNA.,HUMBLE.,0.0190,0.818966,17.424798
7,Bounce Back,Bad and Boujee (feat. Lil Uzi Vert),0.0169,0.563333,15.958451
8,Mask Off,XO TOUR Llif3,0.0163,0.515823,15.630993
2,Mask Off,HUMBLE.,0.0204,0.645570,13.735524
10,XO TOUR Llif3,Congratulations,0.0179,0.542424,13.229860
0,XO TOUR Llif3,HUMBLE.,0.0204,0.618182,13.152805
1,Mask Off,Congratulations,0.0162,0.512658,12.503859
5,goosebumps,Congratulations,0.0155,0.503247,12.274311
9,goosebumps,HUMBLE.,0.0167,0.542208,11.536336
3,Congratulations,HUMBLE.,0.0214,0.521951,11.105345


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-5000-0.01.csv', los cuales fueron obtenidos de las 5000 primeras canciones con un support de 0.01. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [12]:
frequent_itemsets_5000_01 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-5000-0.01.csv?token=AOMD44D5BJMWDNNBYM56ZZLBUQTV4', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_5000_01.pop(list(frequent_itemsets_5000_01.keys())[0])
rules_5000_01 = generate_association_rules(frequent_itemsets_5000_01,0.5,1.2)
rules_pd_5000_01 = pd.DataFrame(rules_5000_01,columns = [ "Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_5000_01 = rules_pd_5000_01.sort_values('Lift',ascending=False)
rules_pd_5000_01.to_csv("rules_5000_01.csv",header=None)
rules_pd_5000_01.head(10)

,Antecedente,Consecuente,Support,Confidence,Lift
129,Knee Deep (feat. Jimmy Buffett),Toes,0.0102,0.593023,34.478096
130,Toes,Knee Deep (feat. Jimmy Buffett),0.0102,0.593023,34.478096
145,You Was Right,Money Longer,0.0110,0.604396,32.148702
144,Money Longer,You Was Right,0.0110,0.585106,32.148702
140,Play It Again,That's My Kind Of Night,0.0100,0.520833,31.001984
139,That's My Kind Of Night,Play It Again,0.0100,0.595238,31.001984
59,"(Bank Account, XO TOUR Llif3)",Magnolia,0.0102,0.680000,30.088496
133,Santeria,What I Got,0.0108,0.504673,30.040053
132,What I Got,Santeria,0.0108,0.642857,30.040053
61,"(Magnolia, XO TOUR Llif3)",Bank Account,0.0102,0.728571,28.683915


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-2500-0.01.csv', los cuales fueron obtenidos de las 2500 primeras canciones con un support de 0.01. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [13]:
frequent_itemsets_2500_01 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-2500-0.01.csv?token=AOMD44H5U2KAWPTL7G6PH4DBUQQQS', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_2500_01.pop(list(frequent_itemsets_2500_01.keys())[0])
rules_2500_01= generate_association_rules(frequent_itemsets_2500_01,0.5,1.2)
rules_pd_2500_01 = pd.DataFrame(rules_2500_01,columns = [ "Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_2500_01= rules_pd_2500_01.sort_values('Lift',ascending=False)
rules_pd_2500_01.to_csv("rules_2500_01.csv")
rules_pd_2500_01.head(10)


,Antecedente,Consecuente,Support,Confidence,Lift
171,Knee Deep (feat. Jimmy Buffett),Toes,0.0104,0.619048,38.690476
172,Toes,Knee Deep (feat. Jimmy Buffett),0.0104,0.650000,38.690476
29,"(Mask Off, iSpy (feat. Lil Yachty))",Rolex,0.0100,0.609756,35.450936
31,Rolex,"(Mask Off, iSpy (feat. Lil Yachty))",0.0100,0.581395,35.450936
169,What I Got,Santeria,0.0100,0.675676,34.473249
170,Santeria,What I Got,0.0100,0.510204,34.473249
108,That's My Kind Of Night,Play It Again,0.0104,0.702703,33.783784
109,Play It Again,That's My Kind Of Night,0.0104,0.500000,33.783784
17,"(Bank Account, Butterfly Effect)",rockstar,0.0100,0.781250,33.674569
18,"(Bank Account, rockstar)",Butterfly Effect,0.0100,0.781250,33.674569


### **Comentario:**
- Para los tres casos presentados, los umbrales de support, confidence y lift fueron los mismos. El umbral de support se basa en que al menos 1 itemset frecuente debe estar en el 1% de los playlist, por tanto, en el caso del conjunto de 10000 playlist un itemset debe estar por lo menos en 100 playlists , en un conj. de 5000 playlist debe estar por lo menos en 50 y en un conjunto de 2500, debe estar por lo menos en 25, siendo elegido este umbral viendo que son pocos los casos en los que llega al 4% o 5%.
- La obtención de las reglas depende bastante del tamaño de playlists usados, ya que estos afectan los valores de support, confidence y lift. 
- En el caso del umbral de lift, el límite de este fue tomado en 1.2, afectando en casi nada la generación de reglas, dado que la mayoria de estas sobrepasaban por mucho el valor dado. Es así que al sobrepasar 1 en este caso y obtiendose grandes cifras, basandonos en que cuanto más se aleja el valor de lift de 1, más evidencias de que la regla no se debe a un artefacto aleatorio, es decir, mayor la evidencia de que la regla representa un patrón real. Podemos decir que no es casualidad que los consecuentes de cada una de las reglas fueron pura casualidad.

## **Explicar las reglas obtenidas**
---

Explicaremos algunas de las reglas obtenidas del dataset completo, con support 0.01 y 0.015, las cuales seran:

### **Caso 1:**
---

In [14]:
rules_pd_10000_01.head(2)

,Antecedente,Consecuente,Support,Confidence,Lift
92,No Heart,X (feat. Future),0.0103,0.691275,33.72074
91,X (feat. Future),No Heart,0.0103,0.502439,33.72074


En esta regla de las 10000 playlists con un umbral de support de 0.01, no es coincidencia que una canción de ambas haga escuchar la otra, ya que ambas tienen un alto lift. 

***Descripcion de las canciones:***
- **X(feat. Future):** 
> **Artistas:** 21 Savage, Metro Boomin

> **Artista invitado:** Future

> **Álbum:** Savage Mode

> **Fecha de lanzamiento:** 2016

> **Género:** Hip-hop/rap

- **No Heart:**

> **Artistas:** 21 Savage, Metro Boomin

> **Álbum:** Savage Mode

> **Fecha de lanzamiento:** 2016

> **Género:** Hip-hop/rap

Ambas canciones son pertenecientes al género de Hip-hop/rap. Coincidentemente, ambas canciones son partes del mismo álbum, el cual fue lanzado el 2016, por 21 Savage y Metro Boomin, grandes referentes de este género.

Sobre los indicadores obtenidos tenemos que si se escucha X(feat. Future) hay aprox. el 50% de probabilidades que escuchemos No heart, lo que no sucede a la inversa, donde la probabilidad de que si escucho No Heart escuche X(feat. Future), por otro lado, el alto valor de lift evidencia que estas canciones estan muy presentes en el conjunto de datos.

### **Caso 2:**
---

In [15]:
print_rules([list(rules_pd_10000_01.iloc[6])])


 ======= REGLA DE ASOCIACIÓN =======
Magnolia -------- Bank Account
SUPPORT    =  0.0113
CONFIDENCE =  0.565
LIFT       =  24.458874458874458


Esta regla se extrajó de las 10000 playlists, con un umbral de support de 0.01.

***Descripcion de las canciones:***
- **Magnolia:** 
> **Artistas:** Playboi Carti

> **Álbum:** Playboi Carti

> **Fecha de lanzamiento:** 2017

> **Género:** Hip-hop/rap

- **Bank Account:**

> **Artistas:** 21 Savage

> **Álbum:** Issa Album

> **Fecha de lanzamiento:** 2017

> **Género:** Hip-hop/rap

Ambas canciones son pertenecientes al género de Hip-hop/rap. Lanzadas el mismo año por distintos artistas.

Es probable que esta relación de causalidad se deba a que ambas canciones fueron lanzadas el mismo año, por otro lado el alto valor de lift muestra que ambas canciones se encuentran presentes en muchos playlist, y que en la mayoría de veces ambas canciones a parecen juntas, por lo que podríamos que decir que se sigue un patrón respecto a la música rap, como se mostró en el caso 1 también.

### **Caso 3:**
---

In [16]:
print_rules([list(rules_pd_10000_01.iloc[9])])

 ======= REGLA DE ASOCIACIÓN =======
('HUMBLE.', 'XO TOUR Llif3') -------- DNA.
SUPPORT    =  0.0102
CONFIDENCE =  0.5
LIFT       =  21.551724137931036


Esta regla se extrajo de los 10000 playlists con un umbral de support de 0.01.

***Descripcion de las canciones:***
- **Humble:** 
> **Artistas:** Kendrick Lamar

> **Álbum:** DAMN

> **Fecha de lanzamiento:** 2017

> **Género:** Hip-hop/rap

- **XO TOUR Llif3:**

> **Artistas:**  Lil Uzi Vert

> **Álbum:**  Luv Is Rage 2

> **Fecha de lanzamiento:** 2017

> **Género:** Hip-hop/rap

- **DNA.:**

> **Artistas:**  Kendrick Lamar

> **Álbum:**  Damn

> **Fecha de lanzamiento:** 2017

> **Género:** Hip-hop/rap

Esta regla de 2 a 1, demuestra el patrón en elección de temas musicales, ya que las 3 canciones pertenecen al mismo género. Por otro lado la primera canción del antecedente, tiene una relación muy estrecha con su consecuente ya que pertenecen al mismo album.

La confianza esta en el límite de nuestro umbral, sin embargo puedo dejarnos ver que hay un 50% de probabilidad que ocurra esta regla. 

### **Caso 4:**
---

In [17]:
print_rules([list(rules_pd_10000_015.iloc[1])])

 ======= REGLA DE ASOCIACIÓN =======
Bounce Back -------- Bad and Boujee (feat. Lil Uzi Vert)
SUPPORT    =  0.0169
CONFIDENCE =  0.5633333333333334
LIFT       =  15.958451369216244




Esta regla se extrajó de las 10000 primeras playlists con un umbral de support de 0.015.

***Descripcion de las canciones:***

- **Bounce Back:**

> **Artistas:** Big Sean

> **Álbum:** I Decided

> **Fecha de lanzamiento:** 2017

> **Género:** Hip-hop/rap

- **Bad and Boujee (feat. Lil Uzi Vert):**

> **Artistas:**  Migos, Lil Uzi Vert

> **Álbum:**  Culture

> **Fecha de lanzamiento:** 2016

> **Género:** Hip-hop/rap

Las canciones pertenecientes a esta regla tienen una mélodía similar, dado el género al que pertenecen.  Además fueron lanzadas con fechas próximas lo cual influye en que las personas escuchen ambas. Por otro lado el alto valor de lift, para los parámetros tomados para generar esta regla, refleja que no es casualidad que se escuche Bad and Boujee después de Bounce Back. Así mismo, indagando un poco más sobre los artistas (Big Sean y Migos) tienen canciones que hicieron en colaboración. Lo cuál influye en la relación que tienen.


### **De forma resumida, a partir de las reglas generadas se ve que en el conjunto de datos completo, hay una gran presencia del género Hip-hop/rap en general, y el patrón se establece en la coincidencia en la elección de temas del mismo género. Por otro lado, si bien si fuera el caso de que el umbral de confidence subiría, podríamos probar que la presencia de estas relaciones serían algo comunes. Ya que no es casualidad la elección de canciones del mismo año o género dentro de un playlist.**